In [1]:
import sys
import os

sys.path.append(os.path.abspath("../"))

In [2]:
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingOneHot
from decomposition.qhyper.algorithm import WorkflowDecompositionQHyperAdapter
from decomposition.qhyper.solver import DecomposedWorkflowSchedulingSolver, WorkflowSchedulingSolverDecorator
from QHyper.solvers.classical.gurobi import Gurobi

In [3]:
def deadline_as_cpv(tasks_file, machines_file):
    workflow = Workflow(tasks_file, machines_file, 100000)
    mean_times = workflow.time_matrix.mean(axis=1).to_dict()

    def path_load(p):
        return sum([mean_times[t] for t in p])

    return int(max(path_load(p) for p in workflow.paths))

### Load workflow

In [4]:
tasks_file = "resources/workflows/1000genome-chameleon-2ch-250k-001.json"
machines_file = "resources/machines/linear_smaller_diff.json"
deadline = deadline_as_cpv(tasks_file, machines_file)

workflow = Workflow(tasks_file, machines_file, deadline)

In [5]:
print(len(workflow.wf_instance.workflow.tasks))

82


### Prepare solver with SPWD

In [6]:
division = WorkflowDecompositionQHyperAdapter(workflow).decompose(max_subgraph_size=20)
problems = map(lambda w: WorkflowSchedulingOneHot(w), division.workflows)
solvers = map(lambda p: WorkflowSchedulingSolverDecorator(Gurobi(p)), problems)
solver = DecomposedWorkflowSchedulingSolver(list(solvers), division)

### Solve with decomposition

In [7]:
solution = solver.solve()

In [8]:
print(f"Decomposed solution cost={solution.cost}, time={solution.time}")

Decomposed solution cost=86995.05678959998, time=3780.8972262400002


### Compare without decompistion

In [9]:
full_solution = WorkflowSchedulingSolverDecorator(Gurobi(WorkflowSchedulingOneHot(workflow))).solve()

In [10]:
print(f"Full solution cost={full_solution.cost}, time={full_solution.time}")

Full solution cost=81123.84144480001, time=3905.40979392
